In [29]:
from numpy import *

In [30]:
def loadDataSet(fileName):
    dataMat =[]
    fr = open(fileName)
    for line in fr.readlines():
        curLine = line.strip().split("\t")
        fltLine = list(map(float, curLine))
        dataMat.append(fltLine)
    return dataMat

In [31]:
def distEuclid(vecA, vecB):
    return sqrt(sum(power(vecA-vecB, 2)))

In [32]:
def randCent(dataSet, k):
    n = shape(dataSet)[1]
    centroids = mat(zeros((k, n)))
    for j in range(n):
        minJ = min(dataSet[:,j])
        rangeJ = float(max(dataSet[:,j])-minJ)
        centroids[:,j] = minJ+rangeJ*random.rand(k,1)
    return centroids

In [38]:
datMat =mat(loadDataSet('testSet.txt'))


In [39]:
print(min(datMat[:,0]))

[[-5.379713]]


In [40]:
min(datMat[:,1])

matrix([[-4.232586]])

In [41]:
max(datMat[:,1])

matrix([[5.1904]])

In [44]:
randCent(datMat, 2)

C:\Users\harsh\AppData\Local\Temp\ipykernel_3448\4043334818.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  rangeJ = float(max(dataSet[:,j])-minJ)


matrix([[-5.20617492,  3.59334487],
        [-0.58544225, -1.14041166]])

In [45]:
distEuclid(datMat[0], datMat[1])

5.184632816681332

In [46]:
def kMeans(dataSet, k, distMeas=distEuclid, createCent=randCent):
    m = shape(dataSet)[0]
    clusterAssment = mat(zeros((m, 2)))
    centroids = createCent(dataSet, k)
    clusterChanged = True
    while clusterChanged:
        clusterChanged = False
        for i in range(m):
            minDist = inf
            minIndex = -1
            for j in range(k):
                distJI = distMeas(centroids[j,:], dataSet[i,:])
                if distJI < minDist:
                    minDist = distJI
                    minIndex = j
            if clusterAssment[i,0] != minIndex:
                clusterChanged = True
            clusterAssment[i,:] = minIndex, minDist**2
        print(centroids)
        for cent in range(k):
            ptsInClust = dataSet[nonzero(clusterAssment[:,0].A==cent)[0]]
            centroids[cent,:] = mean(ptsInClust, axis=0)
    return centroids, clusterAssment

In [47]:
myCentroids, clustAssign = kMeans(datMat, 4)

[[ 4.10559028 -3.77129192]
 [ 3.13353451 -3.17444068]
 [ 0.12046373  2.22984949]
 [ 3.97377477 -0.67400863]]
[[ 4.079985   -3.255096  ]
 [ 0.73705233 -3.30043076]
 [-1.19044156  1.53242858]
 [ 3.942136    0.50598643]]
[[ 3.29921309 -3.11143536]
 [-2.071183   -3.04115417]
 [-1.86350681  2.50511937]
 [ 3.08014861  2.43996456]]
[[ 2.926737   -2.70147753]
 [-3.19984738 -2.96423548]
 [-2.46154315  2.78737555]
 [ 2.6265299   3.10868015]]
[[ 2.80293085 -2.7315146 ]
 [-3.38237045 -2.9473363 ]
 [-2.46154315  2.78737555]
 [ 2.6265299   3.10868015]]


C:\Users\harsh\AppData\Local\Temp\ipykernel_3448\4043334818.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  rangeJ = float(max(dataSet[:,j])-minJ)


In [51]:
def biKMeans(dataSet, k, distMeas=distEuclid):
    m = shape(dataSet)[0]
    clustAssment = mat(zeros((m,2)))
    centroid0 = mean(dataSet, axis=0).tolist()[0]
    centList = [centroid0]
    for j in range(m):
        clustAssment[j,1] = distMeas(mat(centroid0), dataSet[j,:])**2
    while (len(centList)<k):
        lowestSSE = inf
        for i in range(len(centList)):
            ptsInCurCluster = dataSet[nonzero(clustAssment[:,0].A == i)[0],:]
            centroidMat, splitClustAs = kMeans(ptsInCurCluster, 2, distMeas)
            sseSplit = sum(splitClustAs[:,1])
            sseNotSplit = sum(clustAssment[nonzero(clustAssment[:,0].A != i)[0], 1])
            print(f'SSE split, and not split:{sseSplit, sseNotSplit}')
            if (sseSplit+sseNotSplit) < lowestSSE:
                bestSentToSplit = i
                bestNewCents = centroidMat
                bestClustAss = splitClustAs.copy()
                lowestSSE = sseSplit+sseNotSplit
        bestClustAss[nonzero(bestClustAss[:,0].A==1)[0], 0] = len(centList)
        bestClustAss[nonzero(bestClustAss[:,0].A==0)[0], 0] = bestSentToSplit
        print(f'Best Sent to split is{bestSentToSplit}')
        print(f'Length of best clust Ass is :{len(bestClustAss)}')
        centList[bestSentToSplit] = bestNewCents[0,:]
        centList.append(bestNewCents[1,:])
        clustAssment[nonzero(clustAssment[:,0].A == bestSentToSplit)[0],:] = bestClustAss
    return (centList), clustAssment


In [49]:
datMat3 = mat(loadDataSet('testSet2.txt'))

In [52]:
centList, myNewAssMents = biKMeans(datMat3, 3)

[[-1.49554016  0.40187742]
 [-2.48627235  2.76930952]]
[[ 0.99677359 -0.73477953]
 [-1.47714714  3.46545543]]
[[ 0.93680474 -0.87084665]
 [-1.3277349   3.46607079]]
[[ 0.63042504 -1.33843332]
 [-0.84735206  3.46862312]]
[[ 0.02053813 -2.21845543]
 [-0.26853357  3.36606168]]
[[-0.32150057 -2.62473743]
 [-0.06953469  3.29844341]]
[[-0.45965615 -2.7782156 ]
 [-0.00675605  3.22710297]]
SSE split, and not split:(453.0334895807502, 0.0)
Best Sent to split is0
Length of best clust Ass is :60
[[ 0.50713355 -2.43499024]
 [-1.28667346 -1.20546279]]
[[-0.05200457 -3.16610557]
 [-1.41084317 -1.873139  ]]
[[-7.11923077e-04 -3.21792031e+00]
 [-1.31198114e+00 -1.96162114e+00]]
[[ 0.07973025 -3.24942808]
 [-1.26873575 -2.07139688]]
[[ 0.19848727 -3.24320436]
 [-1.26405367 -2.209896  ]]
[[ 0.2642961 -3.3057243]
 [-1.1836084 -2.2507069]]
[[ 0.35496167 -3.36033556]
 [-1.12616164 -2.30193564]]
SSE split, and not split:(12.753263136887313, 423.8762401366249)
[[-4.32974472  4.39022542]
 [ 3.2227497   1.5417

C:\Users\harsh\AppData\Local\Temp\ipykernel_3448\4043334818.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  rangeJ = float(max(dataSet[:,j])-minJ)


In [53]:
centList

[matrix([[-0.45965615, -2.7782156 ]]),
 matrix([[-2.94737575,  3.3263781 ]]),
 matrix([[2.93386365, 3.12782785]])]

In [57]:
import urllib3
import json

In [64]:
import urllib.parse
import urllib.request
import json

def geoGrab(stAddress, city):
    apiStem = 'http://where.yahooapis.com/geocode?'  # Base URL for the geocoding API
    params = {
        'flags': 'J',  # JSON return type
        'appid': 'aaa0VN6k',
        'location': f'{stAddress} {city}'
    }
    url_params = urllib.parse.urlencode(params)  # Use urllib.parse.urlencode
    yahooApi = apiStem + url_params
    print(yahooApi)
    
    # Open the URL and retrieve the response
    with urllib.request.urlopen(yahooApi) as c:
        return json.loads(c.read().decode())  # Ensure proper decoding


from time import sleep
def massPlaceFind(fileName):
    fw = open('places.txt', 'w')
    for line in open(fileName).readlines():
        line = line.strip()
        lineArr = line.split('\t')
        retDict = geoGrab(lineArr[1], lineArr[2])
        if retDict['ResultSet']['Error'] == 0:
            lat = float(retDict['ResultSet']['Results'][0]['latitude'])
            lng = float(retDict['ResultSet']['Results'][0]['longitude'])
            print("%s\t%f\t%f" % (lineArr[0], lat, lng))
            fw.write('%s\t%f\t%f\n' % (line, lat, lng))
        else: print("error fetching")
        sleep(1)
    fw.close()
    

In [66]:
def distSLC(vecA, vecB):#Spherical Law of Cosines
    a = sin(vecA[0,1]*pi/180) * sin(vecB[0,1]*pi/180)
    b = cos(vecA[0,1]*pi/180) * cos(vecB[0,1]*pi/180) * \
                      cos(pi * (vecB[0,0]-vecA[0,0]) /180)
    return arccos(a + b)*6371.0 #pi is imported with numpy

import matplotlib
import matplotlib.pyplot as plt
def clusterClubs(numClust=5):
    datList = []
    for line in open('places.txt').readlines():
        lineArr = line.split('\t')
        datList.append([float(lineArr[4]), float(lineArr[3])])
    datMat = mat(datList)
    myCentroids, clustAssing = biKmeans(datMat, numClust, distMeas=distSLC)
    fig = plt.figure()
    rect=[0.1,0.1,0.8,0.8]
    scatterMarkers=['s', 'o', '^', '8', 'p', \
                    'd', 'v', 'h', '>', '<']
    axprops = dict(xticks=[], yticks=[])
    ax0=fig.add_axes(rect, label='ax0', **axprops)
    imgP = plt.imread('Portland.png')
    ax0.imshow(imgP)
    ax1=fig.add_axes(rect, label='ax1', frameon=False)
    for i in range(numClust):
        ptsInCurrCluster = datMat[nonzero(clustAssing[:,0].A==i)[0],:]
        markerStyle = scatterMarkers[i % len(scatterMarkers)]
        ax1.scatter(ptsInCurrCluster[:,0].flatten().A[0], ptsInCurrCluster[:,1].flatten().A[0], marker=markerStyle, s=90)
    ax1.scatter(myCentroids[:,0].flatten().A[0], myCentroids[:,1].flatten().A[0], marker='+', s=300)
    plt.show()